In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

train_df=pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test_df=pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
X_train_full=train_df
X_train_full.head(10)

In [ ]:
y_train_full=X_train_full.iloc[:,0]
X_train_full=X_train_full.iloc[:,1:]


In [ ]:
y_train_full.head(10)

In [ ]:
y_train_full.value_counts().sort_index()

Training data misses label "9" in the sign language.

In [ ]:
X_train_full.shape

Since both training and testing data misses the label "9",there will be no problem.

When splitting the training data and validation data,all distinct labels should be considered.

In [ ]:
X_valid,X_train=X_train_full.iloc[:5491,:]/255,X_train_full.iloc[5491:,:]/255
y_valid,y_train=y_train_full.iloc[:5491],y_train_full.iloc[5491:]


In [ ]:
(y_valid.value_counts()/len(y_valid)).sort_index()

In [ ]:
(y_train.value_counts()/len(y_train)).sort_index()

A descent sampling to generate training and validation set.

**All data are converted into numpy arrays for ease of visualization of the sign language picture plots.

In [ ]:
y_valid=y_valid.to_numpy()
y_train=y_train.to_numpy()

X_train=X_train.to_numpy()
X_valid=X_valid.to_numpy()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_train[0].reshape(28,28),interpolation="gaussian")
plt.axis("off")
plt.show()

In [ ]:
y_train[0] # The above figure is a 7 in sign language.

Let us plot to see all the different sign languages.

In [ ]:
def index_sign_labels(target):
    labels_index=[None]*25   # to store index of individual labels
    for i in range(len(target)):
        labels_index[target[i]]=i
    return labels_index

In [ ]:
idx=index_sign_labels(y_train)

n_rows=5
n_columns=5

i=0
plt.figure(figsize=(15,12))
for rows in range(n_rows):
    for columns in range(n_columns):
        if i!=9:
            index=n_columns*rows+columns
            plt.subplot(n_rows,n_columns,index+1)
            plt.imshow(X_train[idx[i]].reshape(28,28),interpolation="gaussian")
            plt.axis("off")
            plt.title(i)
        i+=1

In [ ]:
import tensorflow as tf
from tensorflow import keras

**Early stopping with patience limit 10 epochs for capturing the best weights.

In [ ]:
early_stopping_cb=keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

model=keras.models.Sequential([keras.layers.Dense(128,activation='relu',input_shape=[784]),
                               keras.layers.Dense(100,activation='relu'),
                               keras.layers.Dense(100,activation='relu'),
                               keras.layers.Dense(100,activation='relu'),
                               keras.layers.Dense(25,activation='softmax')])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [ ]:
history=model.fit(X_train,y_train,epochs=150,validation_data=(X_valid,y_valid),callbacks=[early_stopping_cb])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(12,8))
plt.grid(True)
plt.ylim(0,1.2)
plt.xlabel("Epochs",fontsize=14)
plt.ylabel("Loss / Accuracy",fontsize=14)
plt.show()

In [ ]:
X_test_full=test_df

X_test=X_test_full.iloc[:,1:]/255
y_test=X_test_full.iloc[:,0]

X_test=X_test.to_numpy()
y_test=y_test.to_numpy()

model.evaluate(X_test,y_test)

Accuracy is not as high as expected.Let us see pick first 10 samples of test set and see the accuracy on them.

**The output of the neural network contains softmax activation function.It produces probablities of all the class.

In [ ]:
size=len(y_test)
n_class=24

X_new=X_test[-10:,:]
output_probs=model.predict(X_new)
y_pred=[]

for i in range(10):
    m=output_probs[i][0]
    x=0
    for j in range(1,n_class):
        if output_probs[i][j]>m:
            m=output_probs[i][j]
            x=j
    y_pred.append(x)

y_pred

In [ ]:
y_test[-10:]

4 out of last 10 instances in the test set are incorrectly classified.

In [ ]:
y_pred=np.array(y_pred)

In [ ]:
acc=sum(y_pred==y_test)/len(y_test)
acc

Inspite of having good validation accuracy,the model does not perform exceptionally in the test set.Let us see the figures in the test set,to check if they are different from training instances. 

Let us plot the test set figures.

In [ ]:
idx=index_sign_labels(y_test)

n_rows=5
n_columns=5

i=0
plt.figure(figsize=(15,12))
for rows in range(n_rows):
    for columns in range(n_columns):
        if i!=9:
            index=n_columns*rows+columns
            plt.subplot(n_rows,n_columns,index+1)
            plt.imshow(X_train[idx[i]].reshape(28,28),interpolation="gaussian")
            plt.axis("off")
            plt.title(i)
        i+=1

****It is clearly evident that the training and testing instances do not match exactly for many target values.Hence is the failure of the model in test set inspite of having excellent validation accuracies inside the training set.****

****If you like my notebook,please upvote and also comment for further improvements.